In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

In [3]:
df = pd.read_json('ai_papers.json')

In [4]:
# Preprocessing the dataset
df['abstract'] = df['abstract'].apply(clean_txt)

In [5]:
# Looking at the cleaned dataset
df.head()

,id,title,category,abstract,version
0,0704.0047,Intelligent location of simultaneously active ...,cs.NE cs.AI,intelligent acoustic emission locator describe...,"Sun, 1 Apr 2007 13:06:50 GMT"
1,0704.0050,Intelligent location of simultaneously active ...,cs.NE cs.AI,part describe intelligent acoustic emission lo...,"Sun, 1 Apr 2007 18:53:13 GMT"
2,0704.0304,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,paper discuss benefit describe world informati...,"Wed, 13 Oct 2010 19:49:16 GMT"
3,0704.0671,Learning from compressed observations,cs.IT cs.LG math.IT,problem statistical learn construct predictor ...,"Thu, 5 Apr 2007 02:57:15 GMT"
4,0704.0954,Sensor Networks with Random Links: Topology De...,cs.IT cs.LG math.IT,sensor network practice communication among se...,"Fri, 6 Apr 2007 21:58:52 GMT"


In [6]:
# Tfidf Method
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf_corpus = tfidf_vectorizer.fit_transform((df['abstract'])) #fitting and transforming the vector
tfidf_corpus

<91496x78832 sparse matrix of type '<class 'numpy.float64'>'
	with 6652009 stored elements in Compressed Sparse Row format>

In [7]:
user_query = "face recognition deep learning"

In [8]:
user_query = clean_txt(user_query)

In [9]:
user_query

'face recognition deep learn'

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform([user_query])
cos_similarity_tfidf = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_corpus)

In [11]:
output2 = list(cos_similarity_tfidf)

In [17]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['title', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'title'] = df['title'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation

In [18]:
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:10]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top,df, list_scores)

,title,score
0,HyperFaceNet: A Hyperspectral Face Recognition...,0.59844
1,Does Generative Face Completion Help Face Reco...,0.566985
2,On effective human robot interaction based on ...,0.552937
3,Efficient Decision-based Black-box Adversarial...,0.533845
4,Face representation by deep learning: a linear...,0.519453
5,Face Recognition by Fusion of Local and Global...,0.506273
6,Examplers based image fusion features for face...,0.504076
7,Boosting Deep Face Recognition via Disentangli...,0.500588
8,Look Across Elapse: Disentangled Representatio...,0.498182
9,Comparing Robustness of Pairwise and Multiclas...,0.493181
